In [4]:
# Imports
import numpy as np
import altair as alt
import pydot
from IPython.display import display, SVG, clear_output, HTML
import matplotlib.pyplot as plt
from pydrake.all import *
# from manipulation import running_as_notebook
# from manipulation.meshcat_cpp_utils import(
#     StartMeshcat, MeshcatJointSlidersThatPublish)
# from manipulation.scenarios import AddMultibodyTriad

# Import pygmo for optimisation
import pygmo as pg
import pandas as pd

In [5]:
# GLOBAL PROBLEM SETUP

# Reference: https://wiki.ros.org/pr2_controller_manager/safety_limits
# Defining the actuator box
class ActuatorBox(LeafSystem):
    # Input values are for motor and NOT for the joint
    def __init__(self, tau_max_m, omega_max_m, gear_ratio, k_velocity):
        LeafSystem.__init__(self)
        # Both the values are motor specific NOT joint specific
        # Get joint max torque
        self.tau_max_j = tau_max_m*gear_ratio
        self.k_vel = k_velocity
        # Convert RPM of motor to joint velocity in rad/sec
        self.omega_max_j = omega_max_m*(2*np.pi/60)/gear_ratio
        self.DeclareVectorInputPort('commanded_torque', BasicVector(na))
        self.DeclareVectorInputPort('joint_state', BasicVector(2*na))
        self.DeclareVectorOutputPort('realised_torque', BasicVector(na), self.OutputJointTorque)

    def OutputJointTorque(self, context, output):
        # Get the state values
        q = self.EvalVectorInput(context, 1).get_value()
        qvel = q[na:]
        u = self.EvalVectorInput(context, 0).get_value()
        # Compute the instantaneous torque limit
        # For the AK-70-10 motors
        kv = self.k_vel
        u_new = np.copy(u)
        # Using URDF safety controller rules
        for i in range(len(u)):
            if qvel[i]>=0 and u[i]>=0:
                u_bound = -kv*(qvel[i]-self.omega_max_j)
                u_new[i] = np.min([u[i], u_bound, self.tau_max_j])
            if qvel[i]>=0 and u[i]<=0:
                u_bound = -self.tau_max_j
                u_new[i] = np.max([u[i], u_bound])
            if qvel[i]<=0 and u[i]>=0:
                u_bound = self.tau_max_j
                u_new[i] = np.min([u[i], u_bound])
            if qvel[i]<=0 and u[i]<=0:
#                 u_bound = -kv*(qvel[i]+self.omega_max_j)
                u_bound = -(-kv*(np.abs(qvel[i])-self.omega_max_j))
                u_new[i] = np.max([u[i], u_bound, -self.tau_max_j])
#             print(qvel[i], u[i], u_bound, u_new, '\n')
        output.SetFromVector(u_new)

# Define the number of actuators
na = 7

# Task definition
# Home position
q0 = np.array([3.04, 0.07, 2.47, -0.48, 2.06, -0.89, 0.62])
# Pre defined joint locations as long as design isnt changed
qi = np.array([-2.44638919,0.23876803,1.59048134,0.09794963,3.00620177,-0.84260006,-1.43429923])
qint = np.array([-2.38357708,0.08599855,2.14263713,-0.47696921,2.73985728,-1.57634596,-1.86168229])
qd = np.array([-2.53725333,-0.32203878,1.97857906,-0.12785049,3.03252099,-0.9631777,-2.07610007])


# Pick location
carti = np.array([-0.33, 0.07, 0.8])
# Enforcing the end-effector to be parallel with the table
orii = np.array([-1.57, 0, 0])
Ri = RotationMatrix(RollPitchYaw(orii))
# Place location
cartd = np.array([-0.03, 0.07, 0.8])
# Intermediate location
cartint = (cartd+carti)/2+np.array([0,-0.05,0.2])

# Simulation settings
time_step = 1e-4
sim_time = 5
error_threshold = 5e-3

# Define design variables here
# Assumed values
tau_mu = 5.3e-3
b0 = 1.5e-5
# AK70-10 parameters
# gear ratio
gear_ratio = 10
# motor design variables
tau_max_m = 12/gear_ratio # Nm
omega_max_m = 475*gear_ratio   # RPM
Kt = 0.095        # Nm/A
Kv = 100          # RPM/V
Km = 0.19         # Nm/sqrt(W)
# Computed matrix for joule loss
K_joule = 1/np.sqrt(Km)*np.identity(na)
k_velocity = 8.33/(75*2*np.pi/60) # Nm/(rad/s)

# Control attributes
Kp = 15

case = "baseline_control"

if case=="baseline_control":
    # For baseline-control study
    pre_val_vals = [tau_mu, b0, Km, Kt, Kv, k_velocity, tau_max_m, omega_max_m, gear_ratio]
    pre_val_names = ["tau_mu", "b0", "Km", "Kt", "Kv", "k_velocity", "tau_max_m", "omega_max_m", "gear_ratio"]
    # Define a dictionary for ease of usage and modulartiy of the dvs
    pre_val = dict(zip(pre_val_names, pre_val_vals))
    dv_names = ["Kp", "Kd"]


print(pre_val, dv_names)

{'tau_mu': 0.0053, 'b0': 1.5e-05, 'Km': 0.19, 'Kt': 0.095, 'Kv': 100, 'k_velocity': 1.0606085407643906, 'tau_max_m': 1.2, 'omega_max_m': 4750, 'gear_ratio': 10} ['Kp', 'Kd']


In [6]:
def compute_QoI(pre_val, dv_names, sample_dv):
    # Construct the master dv list
    dv = {**pre_val, **dict(zip(dv_names, sample_dv))}
    # Enter the robot path and finalise sim settings
    urdf_path = './urdfs/DIVAR2V/urdf/DIVAR2V_eef.urdf'
    # Construct the sim diagram
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=time_step)
    model = Parser(plant, scene_graph).AddModelFromFile(urdf_path, 'diva_robot')
    X_R = RigidTransform(RotationMatrix.MakeYRotation(0), np.array([-0.1, 0.5, 1]))
    plant.WeldFrames(plant.world_frame(), plant.get_body(plant.GetBodyIndices(model)[0]).body_frame(), X_R)
    # Due to the way the problem is formulated, spheres and the table can be safely ignored
    plant.Finalize()
    actuator = builder.AddSystem(ActuatorBox(dv['tau_max_m'], dv['omega_max_m'], dv['gear_ratio'], dv['k_velocity']))
    
    # Initialise controller params, for now limited to Kp and Kd search
    Kp = np.full(na, dv['Kp'])
    Ki = np.full(na, 0)
    Kd = np.full(na, dv['Kd'])
    
    iiwa_controller = builder.AddSystem(InverseDynamicsController(plant, Kp, Ki, Kd, False))
    iiwa_controller.set_name("iiwa_controller");
    # Complete connections
    builder.Connect(plant.get_state_output_port(model),
                iiwa_controller.get_input_port_estimated_state())
    builder.Connect(iiwa_controller.get_output_port_control(), 
                actuator.get_input_port(0))
    builder.Connect(plant.get_state_output_port(model),
                actuator.get_input_port(1))
    builder.Connect(actuator.get_output_port(), plant.get_actuation_input_port())
    
    # Connecting a data logger
    # Commanded torque
    logger1 = LogVectorOutput(iiwa_controller.get_output_port_control(), builder)
    # Joint state
    logger2 = LogVectorOutput(plant.get_state_output_port(model), builder)
    # Realised torque
    logger3 = LogVectorOutput(actuator.get_output_port(), builder)

    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyMutableContextFromRoot(context)
    gripper_frame = plant.GetBodyByName("eef").body_frame()
    
    # Now computing the QoIs
    # WORKSPACE
    # Workspace is always satisfied since its the same robot
    workspace = 0
    # DEFLECTION
    # Deflection is not applicable as we consider only rigid components
    deflection = 0
    # APPLIED FORCE
    # Applied force will also remain the same given that the tau_max is fixed
    applied_force = 0
    # PAYLOAD
    # same as applied force
    payload = 0
    # Setup the simulation
    xd = np.hstack((qint, 0*qint))
    plant.SetPositions(plant_context, qi)
    iiwa_controller.GetInputPort('desired_state').FixValue(
    iiwa_controller.GetMyMutableContextFromRoot(context), xd);
    # Simulation
    simulator = Simulator(diagram, context)
    simulator.AdvanceTo(sim_time);
    
    # Now we want to get the simulation data, i.e., 
    # the torque applied, final time of the joint variables
    log1 = logger1.FindLog(context)
    log2 = logger2.FindLog(context)
    log3 = logger3.FindLog(context)
    
    error = np.abs(log2.data()[:na,:].transpose() - np.tile(qint,(log3.data().shape[1],1)))
    e_norm = np.array([np.linalg.norm(val) for val in error])
    for i in range(len(e_norm)):
        if np.all(e_norm[i:]<error_threshold):
            break
    final_error = e_norm[-1]
    # CYCLE TIME
    ts = log2.sample_times()[i]
    
    omega_j = []
    for i in range(log2.data().shape[1]):
        omega_j.append(log2.data()[:,i][na:])
    omega_j = np.array(omega_j).transpose()
    omega_m = omega_j*gear_ratio
    # Convert the motor speed to RPM
    omega_m_rpm = omega_m*60/(2*np.pi)
    # Motor torque realised
    tau_m = log3.data()/gear_ratio
    
    # Total power losses
    inst_friction_power = []
    inst_joule_power = []
    for i in range(len(log3.sample_times())):
        omega_inst = omega_m[:, i]
        tau_f = dv['tau_mu']*np.sign(omega_inst)+dv['b0']*omega_inst
        P_f = np.dot(tau_f, omega_inst)
        inst_friction_power.append(P_f)
        tau_t = tau_f+tau_m[:, i]
        P_t = np.dot(K_joule.dot(tau_t), tau_t)
        inst_joule_power.append(P_t)
    
    # POWER LOSSES
    total_friction_loss = np.sum(inst_friction_power)*time_step
    total_joule_loss = np.sum(inst_joule_power)*time_step
    
    results = np.array([workspace, deflection, applied_force, payload, ts, total_friction_loss, total_joule_loss, final_error])
    print(results)
    return results

In [7]:
# test with a sample
sample_dv = [15, 2*np.sqrt(15)]
compute_QoI(pre_val, dv_names, sample_dv)

[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 1.97380000e+00 1.48929578e-01 1.40334040e+01 9.73874246e-08]


array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       1.97380000e+00, 1.48929578e-01, 1.40334040e+01, 9.73874246e-08])

In [8]:
# Setting up a problem
class Baseline_Control:
    def __init__(self):
        pass
    
    def fitness(self,sample_dv):
        print(sample_dv)
        # Write a function to output the QoI values
        results = compute_QoI(pre_val, dv_names, sample_dv)
        # Add weighting factor, K_w
        # workspace, deflection, applied_force, payload, ts, total_friction_loss, total_joule_loss, final_error
        K_w = np.diag([1,1,1,1,1,1,1,1])
        # Quadratic objective meta function with weights
        fitness_val = np.matmul(K_w, results).dot(results)
        fitness_val = results.sum()
        return [fitness_val]
    
    def get_bounds(self):
        # Define bounds on the controller variables
        # Ideally from a csv file like before so that one can easily extend the optimisation problem
        return ([0,0],[100,100])
    
    def get_name(self):
        # get the name of the problem also from the csv sheet
        return "Baseline_Control"

In [9]:
prob = pg.problem(Baseline_Control())
print(prob)
print(prob.get_bounds())

Problem name: Baseline_Control
	C++ class name: pybind11::object

	Global dimension:			2
	Integer dimension:			0
	Fitness dimension:			1
	Number of objectives:			1
	Equality constraints dimension:		0
	Inequality constraints dimension:	0
	Lower bounds: [0, 0]
	Upper bounds: [100, 100]
	Has batch fitness evaluation: false

	Has gradient: false
	User implemented gradient sparsity: false
	Has hessians: false
	User implemented hessians sparsity: false

	Fitness evaluations: 0

	Thread safety: none

(array([0., 0.]), array([100., 100.]))


In [10]:
# Using CMA-ES
algo = pg.algorithm(pg.cmaes(gen = 50, sigma0=0.3))
print(algo)

Algorithm name: CMA-ES: Covariance Matrix Adaptation Evolutionary Strategy [stochastic]
	C++ class name: pagmo::cmaes

	Thread safety: basic

Extra info:
	Generations: 50
	cc: auto
	cs: auto
	c1: auto
	cmu: auto
	sigma0: 0.3
	Stopping xtol: 1e-06
	Stopping ftol: 1e-06
	Memory: false
	Verbosity: 0
	Force bounds: false
	Seed: 3492894556


In [11]:
archi = pg.archipelago(n=16, algo = algo, prob=prob, pop_size=100)
archi.evolve(); archi.wait()

[12.62157326 63.24318134]
[ 0.          0.          0.          0.          5.          0.09287823
 13.39639086  0.44464948]
[46.37543082 93.46586049]
[ 0.          0.          0.          0.          5.          0.13525162
 13.92869062  0.1000419 ]
[ 9.94553599 22.0523563 ]
[ 0.          0.          0.          0.          5.          0.13230939
 13.86884408  0.12317767]
[93.62958027 42.14169887]
[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 2.33750000e+00 1.54151329e-01 1.42424996e+01 9.51307931e-06]
[57.90891946 85.04477691]
[ 0.          0.          0.          0.          5.          0.14301757
 14.01583443  0.03921907]
[43.75924462 66.85944817]
[ 0.          0.          0.          0.          5.          0.14221563
 13.99818136  0.04464928]
[40.68286031 35.99065395]
[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 4.72200000e+00 1.47922174e-01 1.40686853e+01 3.61326757e-03]
[27.88735262 96.3411287 ]
[ 0.          0.          0.          0.          5

[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 2.20960000e+00 3.66524608e-01 1.42346852e+01 2.01183785e-06]
[68.70521954 47.02439437]
[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 3.64900000e+00 1.49782915e-01 1.41343537e+01 6.50636829e-04]
[72.02205775 30.37915386]
[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 2.13670000e+00 1.53847089e-01 1.42164324e+01 2.99491258e-06]
[59.201669   53.97596424]
[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 4.91240000e+00 1.48116403e-01 1.40879323e+01 4.53225550e-03]
[93.29855499 27.61272675]
[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 1.35730000e+00 1.65844390e-01 1.43865012e+01 2.91922269e-09]
[14.30598573 22.24048911]
[ 0.          0.          0.          0.          5.          0.14201397
 13.96561902  0.04521249]
[ 3.71753385 89.07570995]
[ 0.          0.          0.          0.          5.          0.02762307
 12.29485037  0.97930162]
[54.7304238  42.96547544]
[0.00

[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 4.35650000e+00 1.49106700e-01 1.41248993e+01 2.21985275e-03]
[89.80243985 91.7229864 ]
[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 5.00000000e+00 1.47652705e-01 1.40980421e+01 8.60134197e-03]
[83.40393109 46.58762415]
[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 2.95250000e+00 1.51498258e-01 1.41845267e+01 1.09836244e-04]
[42.85746472 33.10012466]
[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 4.09270000e+00 1.48530856e-01 1.40845761e+01 1.46928774e-03]
[54.85125679 26.68884528]
[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 2.47820000e+00 1.51239529e-01 1.41556088e+01 1.74496129e-05]
[ 3.98042828 36.65058352]
[ 0.          0.          0.          0.          5.          0.06148021
 12.90007063  0.7016816 ]
[77.03293937 74.22547419]
[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 5.00000000e+00 1.47986322e-01 1.40971831e+01 6.29792486e-03]
[34.509

[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 1.28720000e+00 1.63915292e-01 1.43429457e+01 7.12339446e-10]
[89.35713379 59.82653755]
[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 3.58660000e+00 1.50344310e-01 1.41617198e+01 5.72917384e-04]
[88.5748623  14.37012202]
[  0.           0.           0.           0.           5.
  31.56917234 100.69994388   6.50405413]
[ 0.99635074 55.37779793]
[ 0.          0.          0.          0.          5.          0.01259464
 12.00583677  1.10265887]
[60.34146164 84.51844448]
[ 0.          0.          0.          0.          5.          0.14383283
 14.02648816  0.03315773]
[92.23032879 48.37681479]
[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 2.76690000e+00 1.52316041e-01 1.42078025e+01 5.90120189e-05]
[42.72951526 10.31148392]
[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 1.10910000e+00 1.65745106e-01 1.40522273e+01 9.94328938e-12]
[76.38965959 58.16601573]
[0.00000000e+00 0.00000000e+0

[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 4.64100000e+00 1.47757396e-01 1.40539985e+01 3.25950548e-03]
[97.87526148 57.10425964]
[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 3.10780000e+00 1.51508537e-01 1.41940097e+01 1.76019463e-04]
[68.90951337 71.66891915]
[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 5.00000000e+00 1.47341179e-01 1.40798259e+01 9.30787516e-03]
[ 5.77221399 77.6040666 ]
[ 0.          0.          0.          0.          5.          0.04558006
 12.62589286  0.83200708]
[45.57835139 61.94449055]
[ 0.          0.          0.          0.          5.          0.14420906
 14.02096752  0.02943125]
[ 7.30861238 18.48562947]
[ 0.          0.          0.          0.          5.          0.12733918
 13.81297839  0.16348575]
[ 9.56336323 17.73966464]
[ 0.          0.          0.          0.          5.          0.13801527
 13.92004604  0.07711622]
[23.30305529 43.47639302]
[ 0.          0.          0.          0.          5. 

...

[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 9.92100000e-01 1.80800050e-01 1.42512109e+01 4.46367711e-13]
[9.18367569 7.35841857]
[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 3.64730000e+00 1.48107912e-01 1.39992538e+01 5.79861162e-04]
[91.32984246 64.23675808]
[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 3.77820000e+00 1.50068085e-01 1.41558159e+01 8.45476587e-04]
[88.90836444 92.11589577]
[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 5.00000000e+00 1.47530244e-01 1.40952641e+01 9.23547612e-03]
[59.26204219 35.13205263]
[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 3.11070000e+00 1.50250227e-01 1.41389888e+01 1.74280525e-04]
[42.99647699 16.6881611 ]
[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 1.79840000e+00 1.51744408e-01 1.41508379e+01 1.87484662e-07]
[44.22492685 88.78489334]
[ 0.          0.          0.          0.          5.          0.13536893
 13.92877421  0.09903107]
[41.89237

[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 5.00000000e+00 1.47977510e-01 1.41081622e+01 7.22731723e-03]
[15.45566558 83.38272443]
[ 0.          0.          0.          0.          5.          0.08886492
 13.33962008  0.47747426]
[92.74654931 91.4471057 ]
[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 5.00000000e+00 1.47947873e-01 1.41053552e+01 7.18510506e-03]
[23.50572443 15.14412612]
[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 3.20440000e+00 1.48490905e-01 1.40565808e+01 2.13785371e-04]
[77.45648853 45.56870942]
[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 3.11510000e+00 1.50932477e-01 1.41676853e+01 1.78425006e-04]
[36.28636014 12.67484308]
[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 1.41920000e+00 1.51460947e-01 1.41358607e+01 8.14313839e-10]
[71.11154246 74.11799547]
[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 5.00000000e+00 1.47344353e-01 1.40813058e+01 9.42245528e-03]
[43.980

[ 0.          0.          0.          0.          5.         26.94512504
 90.1398448   5.62422158]
[58.42459946 59.97187024]
[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 5.00000000e+00 1.47330657e-01 1.40724561e+01 8.63136776e-03]
[96.50996761 69.50985037]
[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 3.87660000e+00 1.49998407e-01 1.41565854e+01 1.01737592e-03]
[6.96508619 9.9286847 ]
[ 0.          0.          0.          0.          5.          0.14399372
 13.95653669  0.02947543]
[62.59437943  9.17397639]
[ 0.          0.          0.          0.          5.         34.74950161
 97.97320351 10.4362286 ]
[19.64541609 54.50118878]
[ 0.          0.          0.          0.          5.          0.12311578
 13.78487     0.1978467 ]
[23.71149131 25.07341618]
[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 5.00000000e+00 1.46735915e-01 1.40268473e+01 9.16203029e-03]
[79.63832517 68.20584059]
[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.0000000

[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 8.48600000e-01 1.59650327e-01 1.40821489e+01 5.27088801e-13]
[79.94478409  3.93750358]
[  0.           0.           0.           0.           5.
  35.55692137 104.06062643  22.8954684 ]
[99.89142885 73.0626332 ]
[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 3.94140000e+00 1.49954018e-01 1.41567922e+01 1.14341245e-03]
[79.93260723 96.78702741]
[ 0.          0.          0.          0.          5.          0.14594312
 14.06456755  0.01882434]
[21.79113768 58.752722  ]
[ 0.          0.          0.          0.          5.          0.12435459
 13.79989996  0.18776657]
[ 9.67995375 74.83050902]
[ 0.          0.          0.          0.          5.          0.06998121
 13.04499751  0.63204362]
[45.17639064 12.40686096]
[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 8.86700000e-01 1.56740237e-01 1.40352784e+01 2.30262639e-13]
[36.80308108 69.74943359]
[ 0.          0.          0.          0.          5. 

[ 0.          0.          0.          0.          5.          0.13991464
 13.96247653  0.06203417]
[17.33189988 57.00098633]
[ 0.          0.          0.          0.          5.          0.11512199
 13.6913987   0.26296761]
[25.62229858 88.51092394]
[ 0.          0.          0.          0.          5.          0.11263906
 13.66534184  0.28321607]
[17.39015819 36.86097458]
[ 0.          0.          0.          0.          5.          0.13369337
 13.89303404  0.11200065]
[ 8.53279326 59.44191758]
[ 0.          0.          0.          0.          5.          0.07526246
 13.12897549  0.58879699]
[74.73957609 42.68677273]
[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 3.01350000e+00 1.51057003e-01 1.41683736e+01 1.32097569e-04]
[64.64865043 25.58705564]
[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 1.95940000e+00 1.54079142e-01 1.42116345e+01 8.83426594e-07]
[70.81922739 18.20382678]
[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 1.04470000e+00 

In [12]:
a = archi.get_champions_f()
a2 = sorted(archi.get_champions_f(), key = lambda x: x[0])[0]
best_isl_idx = [(el == a2).all() for el in a].index(True)
x_best = archi.get_champions_x()[best_isl_idx]
f_best = archi.get_champions_f()[best_isl_idx]
print("Best relaxed solution, x: {}".format(x_best))
print("Best relaxed solution, f: {}".format(f_best))

Best relaxed solution, x: [34.87977586 10.57392095]
Best relaxed solution, f: [15.03969206]


In [ ]:
def gripper_forward_kinematics_example():
    urdf_path = './urdfs/DIVAR2V/urdf/DIVAR2V_eef.urdf'
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=1e-4)
    model = Parser(plant, scene_graph).AddModelFromFile(urdf_path, 'diva_robot')
    # Transform for the robot location
    X_R = RigidTransform(RotationMatrix.MakeYRotation(0), np.array([-0.1, 0.5, 1.25]))
#     X_R = RigidTransform(RotationMatrix(RollPitchYaw([0.8, -0.8, 0])), np.array([-0.1, 0.5, 1]))
    plant.WeldFrames(plant.world_frame(), plant.get_body(plant.GetBodyIndices(model)[0]).body_frame(), X_R)
    # Spawn table
    table = Parser(plant, scene_graph).AddModelFromFile('./urdfs/table/extra_heavy_duty_table_modified.sdf','table')
    # Spawn spherical work piece
    sphere = Parser(plant, scene_graph).AddModelFromFile('./urdfs/helper/sphere.urdf','sphere')
    # Place the sphere at the center of the table
    # Length, width and height and thickness of the table
    # 1.39, 0.762, 0.736, 0.057
    # Sphere radius -- Can be made a design variable
    # 0.15
    # We can sample end-points on the surface of the sphere
    X_R = RigidTransform(RotationMatrix(RollPitchYaw([0, 0, 0])), np.array([0, 0, 0.736]))
    plant.WeldFrames(plant.world_frame(), plant.get_body(plant.GetBodyIndices(sphere)[0]).body_frame(), X_R)

    plant.Finalize()
    
    end_frame = "eef"
    
    # Draw the frames
    for body_name in ["base_link", end_frame]:
        AddMultibodyTriad(plant.GetFrameByName(body_name, model), scene_graph)

    meshcat.Delete()
    meshcat.DeleteAddedControls()

    visualizer = MeshcatVisualizerCpp.AddToBuilder(builder, scene_graph, meshcat)

    diagram = builder.Build()
    context = diagram.CreateDefaultContext()

    plant.SetPositions(plant.GetMyContextFromRoot(context),
                  plant.GetModelInstanceByName("diva_robot"),
                  q0)
    
    gripper = plant.GetBodyByName(end_frame)
    def pose_callback(context):
        pose = plant.EvalBodyPoseInWorld(context, gripper)   ## This is the important line
        print(pose.translation())
        clear_output(wait=True)
        print("gripper position (m): " + np.array2string(
            pose.translation(), formatter={
                'float': lambda x: "{:3.2f}".format(x)}))
        print("gripper roll-pitch-yaw (rad):" + np.array2string(
            RollPitchYaw(pose.rotation()).vector(),
                         formatter={'float': lambda x: "{:3.2f}".format(x)}))
        print("pose rotation: ", pose.rotation())
    sliders = MeshcatJointSlidersThatPublish(meshcat, plant, visualizer, context)
    # sliders.Run()
    sliders.Run(pose_callback)

gripper_forward_kinematics_example()